In [2]:
import pandas as pd

df = pd.read_csv('sentimenttwit_fixed.csv')
print(df.head)

<bound method NDFrame.head of        Sentiment                                              Komen
0             -1  lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
1             -1  kita lanjutkan saja diam ini hingga kau dan ak...
2              1  doa rezeki tak putus inna haa zaa larizquna ma...
3              1  makasih loh ntar kita bagi hasil aku 99 9 sisa...
4             -1  aku tak faham betul jenis orang malaysia yang ...
...          ...                                                ...
10801          1  Jangan membandingkan kehidupanmu dengan kehidu...
10802          0                      Sini uname lu ntar gua follow
10803          1  Apapun yg telah kamu lakukan apapun kesalahanm...
10804          1  3 cara untuk ingat semula apa yang kita dah ha...
10805          1  Semoga Allah permudahkan urusan aku amp orang ...

[10806 rows x 2 columns]>


In [6]:
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import re

nltk.download('stopwords')

factory = StemmerFactory()
stemmer = factory.create_stemmer()

stopwords = set(stopwords.words('indonesian'))

def preprocess(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stopwords]

    return ' '.join(words)

df['clean'] = df['Komen'].apply(preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from sklearn.model_selection import train_test_split

x = df['clean']
y = df['Sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=6468)
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.fit_transform(x_test)

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

model = MultinomialNB()
model.fit(x_train_tfidf, y_train)

y_pred = model.predict(x_test_tfidf)

print("keakuratan : ", accuracy_score(y_test, y_pred))
print("klasifikasi : \n", classification_report(y_test, y_pred))

keakuratan :  0.44912118408880664
klasifikasi : 
               precision    recall  f1-score   support

          -1       0.27      0.15      0.20       560
           0       0.51      0.76      0.61      1101
           1       0.22      0.09      0.13       501

    accuracy                           0.45      2162
   macro avg       0.33      0.34      0.31      2162
weighted avg       0.38      0.45      0.39      2162



In [24]:
import joblib
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tfidf, y_train)

joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']